In [ ]:
!pip install -U pyrogram tgcrypto openai tiktoken ipywidgets

In [ ]:
# API pyrogram
# https://core.telegram.org/api/obtaining_api_id
api_id = 12345678
api_hash = "1234567dfghjrtyur56789tyuihjk"

# API Token Yandex
# https://yandex.cloud/ru/docs/iam/operations/iam-token/create#api_1
OAuthTokenYa="a1_DFGHJjhgfGHJKjhgfGHjhGFfghJHGfghJhgf_A"
# https://yandex.cloud/ru/docs/foundation-models/api-ref/authentication#yandex-account_1
FOLDER_ID = "f456gybt6btgyv67h"

# API Token OpenAI
# https://platform.openai.com/api-keys
api_key="sk-fgHGfGHgfGhGFdfgFDFVBGghfDcvbGfdcVGfGh"

system = "От лица любящего и заботливого мужа Вы продолжаете переписываться с Любимой женой. Напишите, как сильно Вы её любите!"

In [ ]:
# Стандартные библиотеки
from time import sleep
import json

# Сторонние библиотеки
import requests
from pyrogram import Client  # Необходимо для работы с клиентом Pyrogram
from openai import OpenAI
import tiktoken  # Для работы с токенами
import ipywidgets as widgets
from IPython.display import display  # Для отображения содержимого в Jupyter notebook

In [ ]:
async def get_dialogs():
    """
    Асинхронная функция для получения информации о диалогах приватного типа из Telegram аккаунта.

    Использует асинхронный клиент для перебора всех диалогов в аккаунте. Если тип диалога является приватным,
    добавляет информацию о нем в список dialogs, а также собирает и формирует информацию о собеседнике в список info.

    :return: Кортеж из двух элементов:
        - info (list): Список словарей, содержащий информацию о собеседниках:
            - first_name: Имя пользователя
            - last_name: Фамилия пользователя
            - username: Имя пользователя в Telegram
            - id: Идентификатор пользователя
            - date: Дата последнего сообщения
        - dialogs (list): Список объектов диалогов приватного типа.
    """
    info = []
    dialogs = []
    async with Client("my_account", api_id, api_hash) as app:
        async for dialog in app.get_dialogs():
            if ('PRIVATE' in str(dialog.chat.type)):
                dialogs += [dialog]
                info += [
                    {
                        "first_name": dialog.chat.first_name,
                        "last_name": dialog.chat.last_name,
                        "username": dialog.chat.username,
                        "id": dialog.chat.id,
                        "date": dialog.top_message.date,
                    }
                ]
    return info, dialogs


async def get_all_dialog(user_id, n=10, all_dialog=[]):
    """
    Асинхронно получает список сообщений в диалоге с пользователем.

    Эта функция асинхронно извлекает историю сообщений указанного пользователя,
    преобразует её в список словарей и возвращает этот список после сортировки
    по дате отправки сообщений.

    :param user_id: ID пользователя, чью историю сообщений нужно получить.
    :type user_id: int
    :param n: Максимальное количество извлекаемых сообщений (по умолчанию 10).
    :type n: int, optional
    :param all_dialog: Список уже существующих сообщений в диалоге.
                       По умолчанию пустой список. Может быть использован
                       для добавления в него новых сообщений.
    :type all_dialog: list, optional

    :return: Отсортированный список словарей, содержащий информацию о сообщениях.
    :rtype: list

    Каждый словарь в списке содержит:
    - 'from_id': ID пользователя, отправившего сообщение.
    - 'date': Дата и время отправки сообщения.
    - 'text': Текст сообщения или emoji стикера.
    """
    async with Client("my_account", api_id, api_hash) as app:
        
        # Получаем историю сообщений
        async for message in app.get_chat_history(user_id, limit=n):

            text = ''
            if message.text is not None: text = str(message.text)
            if message.sticker is not None: text = str(message.sticker.emoji)
            
            temp = {
                    "from_id": message.from_user.id,
                    "date": message.date,
                    "text": text,
                }

            if not (temp in all_dialog):
                all_dialog += [temp]
                
    all_dialog = sorted(all_dialog, key=lambda x: x['date'])
    
    return all_dialog

def get_chat_history(all_dialog, user_id):
    """
    Получает историю чата с пользователем.

    Эта функция принимает историю всех сообщений в диалоге и идентификатор пользователя,
    затем формирует упорядоченный список диалогов, где каждый элемент списка представляет
    собой тройку: сообщение пользователя, сообщение ассистента и дату последнего сообщения в диалоге.

    Параметры:
    all_dialog : list of dict
        Список словарей, где каждый словарь содержит информацию об одном сообщении.
        Каждое сообщение должно иметь ключи "from_id", "text", и "date".
    user_id : int or str
        Идентификатор пользователя. Используется для определения, от кого каждое сообщение.

    Возврат:
    list of [str | None, str | None, str]
        Возвращает список диалогов, начиная со второго элемента (исключая исходную заготовку),
        где каждый диалог представлен в формате: [сообщение пользователя, сообщение ассистента, дата].
    """
    dialog = [[None, None, None],] # left (user), right (assistant (me)), dt
    for message in all_dialog:
        if (message["from_id"] == user_id) and (dialog[-1][0] is not None):
            # append user many msg
            dialog[-1][0] += "\n" + message["text"]
            dialog[-1][2] = message["date"]
        elif message["from_id"] == user_id:
            # append user one msg
            dialog += [[message["text"], None, message["date"]]]
        elif dialog[-1][1] is not None:
            # append assistant many msg
            dialog[-1][1] += "\n" + message["text"]
            dialog[-1][2] = message["date"]
        else:
            # append assistant one msg
            dialog += [[None, message["text"], message["date"]]]

    return dialog[1:]


def get_history_str(history):
    return '\n'.join([(str(i[0])+'\n'+str(i[1])) for i in history]).replace('None\n','')

def hide_early_messages(history, LIMIT_MAX=2000, model_gpt="gpt-3.5-turbo"):
    history_last = history[:]
    encoding = tiktoken.encoding_for_model(model_gpt)
    while(len(encoding.encode(get_history_str(history_last))) > LIMIT_MAX):
        del history_last[0]
    return history_last


def get_messages_openai(dialog, system):
    history_last = hide_early_messages(dialog)
    messages = []
    messages.append({"role": "system", "content": system})  
    # left (user), right (assistant (me)), dt 
    for i in history_last:
        # print(len(messages), len(history_last))
        if i[0]:
            messages.append({"role": "user", "content": i[0]})
        if i[1] and len(messages)>=2 and len(messages)+1 != len(history_last):
            messages.append({"role": "assistant", "content": i[1]})
            # print(1, i[1])
    return messages

def get_IAM_TOKEN_yandex():
    result = !curl -d "{{\"yandexPassportOauthToken\":\"{OAuthTokenYa}\"}}" "https://iam.api.cloud.yandex.net/iam/v1/tokens"
    
    # Ваша строка
    for i, s in enumerate(result):
        if '{' in s:
            NUM_START = i
    
    json_string = ''.join(result[NUM_START:])
    data_dict = json.loads(json_string)
    
    IAM_TOKEN = data_dict['iamToken']
    return IAM_TOKEN

def get_messages_yandex(dialog, system):
    history_last = hide_early_messages(dialog)
    messages = []
    messages.append({"role": "system", "text": system})  
    # left (user), right (assistant (me)), dt 
    for i in history_last:
        if i[0]:
            messages.append({"role": "user", "text": i[0]})
        if i[1] and len(messages)>=2 and len(messages)+1 != len(history_last):
            messages.append({"role": "assistant", "text": i[1]})
    
    return messages

def make_completion_Ya_2(messages):
    # Асинхронный запрос получения id
    prompt = {
        "modelUri": f"gpt://{FOLDER_ID}/yandexgpt",
        "completionOptions": {
            "stream": False,
            "temperature": 0.6,
            "maxTokens": "2000"
        },
        "messages": messages
    }
    
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync"

    IAM_TOKEN = get_IAM_TOKEN_yandex() #TODO: Добавить проверку по времени 1 час
    headers = {
        "Content-Type": "application/json",
        "x-data-logging-enabled": "false",
        "Authorization": f"Bearer {IAM_TOKEN}"
    }
    
    response = requests.post(url, headers=headers, json=prompt)
    result = response.text
    completion = json.loads(result)
    print(result)
    print(completion['id'])

    # Запрос получения текста по id    
    url = f"https://llm.api.cloud.yandex.net/operations/{completion['id']}"
    headers = {
        # "Content-Type": "application/json",
        "x-data-logging-enabled": "false",
        "Authorization": f"Bearer {IAM_TOKEN}"
    }
    flag = False
    while not flag:  
        sleep(5)
        response = requests.get(url, headers=headers)
        result = response.text
        completion = json.loads(result)
        flag = completion['done']
    
    print(completion['id'])
    
    return completion

def dialog2openai(dialog, system, model="gpt-4-turbo"):
    # OpenAI
    messages = get_messages_openai(dialog, system)

    client = OpenAI(api_key=api_key)
    
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )
    text_chat =  widgets.Textarea(description='user:', value='', layout=widgets.Layout(width='99%'))
    display(text_chat)
    for chunk in stream:
        text_chat.value += chunk.choices[0].delta.content or ""
    return text_chat

def dialog2yandex(dialog, system):
    # Yandex
    messages = get_messages_yandex(dialog, system)

    completion = make_completion_Ya_2(messages)
    text_chat =  widgets.Textarea(description='user:', 
                                    value=completion['response']['alternatives'][0]['message']['text'], 
                                    layout=widgets.Layout(width='99%'))
    display(text_chat)
    return text_chat

In [ ]:
# Выбор диалога
info, all_info = await get_dialogs()
formatted_list = [(f"{item['first_name']} {item['last_name']} (@{item['username']}) - {item['date']}", item['id']) for item in info]
w_info = widgets.Dropdown(
    options=formatted_list,
)
w_info

In [ ]:
all_dialog = []
w_info.get_interact_value()

In [ ]:
# Скачивание последних n сообщений из выбранного диалога
all_dialog = await get_all_dialog(w_info.get_interact_value(), n=10, all_dialog=all_dialog)
dialog = get_chat_history(all_dialog, user_id=w_info.get_interact_value())

In [ ]:
# Ответ от openai
txt = dialog2openai(dialog, system, model="gpt-4-turbo")

In [ ]:
# Ответ от yandex
txt = dialog2yandex(dialog, system)

In [ ]:
# Разделение одного большого текста на короткие сообщения
msgs = ['',]
for ch in txt.value:
    msgs[-1]+=ch
    if ch in ['.', '!', '?']:
        msgs[-1] = msgs[-1].lstrip().rstrip()
        msgs += ['']
msgs = msgs[:-1]

In [ ]:
# Отправка коротких сообщений с интервалом 15 секунд
async with Client("my_account", api_id, api_hash) as app:
    for msg in msgs:
        await app.send_message(w_info.get_interact_value(), msg)
        sleep(15)